### Imports and setup

In [ ]:
import pandas as pd

import yaml
import json
import os
import sys
import warnings
from dotenv import load_dotenv

load_dotenv()
REPO_PATH = os.getenv("REPO_PATH")
sys.path.insert(0, rf'{REPO_PATH}src')
warnings.filterwarnings("ignore")

from utils.main_utils import load_variables
from utils.model_utils import optimize_hyperparameters

### Hyperparameter tuning with Optuna

In [ ]:
# Define feature list and target
var_config = load_variables()

SELECTED_FEATURES = [
    *var_config['BASE'],
    *var_config['TEMPORAL'],
    *var_config['S3']
]

DATA_PARAMS: dict[str, any] = {
    'feature_columns': SELECTED_FEATURES,
    'target_column': 'REALIZED_VOL',
    'test_size': 0.2,
    'val_size': 0.2,
    'scaler_type': 'RobustScaler'
}

TRIAL_PARAMS = {
        'units_first_layer': [16, 32, 64, 128],
        'units_second_layer': [16, 32, 64, 96],
        'l2_strength': [1e-5, 1e-3],
        'learning_rate': [1e-4, 1e-3],
        'batch_size': [16, 32, 64],
        'noise_std': [0.01, 0.1],
        'window_size': [14, 17, 20, 23]
}

# Specify data
FUTURE = 'CLc1'
RNN_TYPE='GRU'
IDENTIFIER = 'S3'
TRIALS = 50

current_dt = pd.Timestamp.now().strftime('%Y.%m.%d_%H.%M')
study_name = f'{IDENTIFIER}_{RNN_TYPE}_{FUTURE}_{current_dt}'

# Find the best hyperparameters using Optuna
best_params = optimize_hyperparameters(
    study_name,
    FUTURE,
    TRIAL_PARAMS,
    DATA_PARAMS,
    RNN_TYPE,
    n_trials=TRIALS,
    n_jobs=6
)

print(f'Best parameters for {RNN_TYPE} model: ', best_params)
# Save best hyperparameters to .json file
with open(f'hyperpm_archive/{study_name}.json', 'w') as file:
    json.dump(best_params, file, indent=4)
